In [1]:
import gurobipy as grb
from gurobipy import GRB
import networkx as nx

In [2]:
import json

space_age = json.load(open("data/space-age.json"))


In [3]:
G = nx.DiGraph()
from collections import defaultdict


machines = defaultdict(set)
for n, f in space_age["furnace"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("furnace", n))

for n, f in space_age["assembling-machine"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("assembling-machine", n))


modules = defaultdict(dict)
for n, m in space_age["module"].items():
    modules[m["category"]][m["name"]] = m["effect"]

print(modules)

max_quality_level = 0
for n, q in space_age["quality"].items():
    max_quality_level = max(max_quality_level, q["level"])

items = set()

for n, r in space_age["recipe"].items():
    if "ingredients" in r:
        items.update((i["type"], i["name"]) for i in r["ingredients"])
    if "results" in r:
        items.update((i["type"], i["name"]) for i in r["results"])


defaultdict(<class 'dict'>, {'speed': {'speed-module': {'speed': 0.2, 'consumption': 0.5, 'quality': -0.1}, 'speed-module-2': {'speed': 0.3, 'consumption': 0.6, 'quality': -0.15}, 'speed-module-3': {'speed': 0.5, 'consumption': 0.7, 'quality': -0.25}}, 'efficiency': {'efficiency-module': {'consumption': -0.3}, 'efficiency-module-2': {'consumption': -0.4}, 'efficiency-module-3': {'consumption': -0.5}}, 'productivity': {'productivity-module': {'productivity': 0.04, 'consumption': 0.4, 'pollution': 0.05, 'speed': -0.05}, 'productivity-module-2': {'productivity': 0.06, 'consumption': 0.6, 'pollution': 0.07, 'speed': -0.1}, 'productivity-module-3': {'productivity': 0.1, 'consumption': 0.8, 'pollution': 0.1, 'speed': -0.15}}, 'quality': {'quality-module': {'quality': 0.1, 'speed': -0.05}, 'quality-module-2': {'quality': 0.2, 'speed': -0.05}, 'quality-module-3': {'quality': 0.25, 'speed': -0.05}}})


In [4]:

from itertools import chain, combinations_with_replacement

all_crafting_variables = set()
all_item_variables = set()

ingredients_to_recipes = defaultdict(set)
products_to_recipes = defaultdict(set)


def get_effect(chosen_modules):
    effects = {
        "consumption": 1.0,
        "speed": 1.0,
        "productivity": 0.0,
        "pollution": 1.0,
        "quality": 0.0,
    }

    for m in chosen_modules:
        module = space_age["module"][m]
        for k in effects:
            effects[k] += module["effect"].get(k, 0)
    
    effects["speed"] = max(effects["speed"], 0.2)
    effects["consumption"] = max(effects["consumption"], 0.2)
    effects["productivity"] = max(effects["productivity"], 0.0)
    effects["pollution"] = max(effects["pollution"], 0.2)
    effects["quality"] = max(effects["quality"], 0.)

    return effects

for _, r in space_age["recipe"].items():
    allow_quality = int(r.get("allow_quality", True))
    for q in range(allow_quality * (max_quality_level)+1):

        for i in r.get("ingredients", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))
        for i in r.get("results", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))


        for m in machines[r.get("category", "crafting")]:
            machine = space_age[m[0]][m[1]]

            if m == ('assembling-machine', 'biochamber'):
                continue

            slots = machine.get("module_slots", 0)
            possible_modules_categories = machine.get("allowed_module_categories", modules.keys())
            possible_modules = set().union(*(set(modules[c].keys()) for c in possible_modules_categories))

            possible_modules = {m for m in possible_modules if space_age["module"][m].get("tier", 1) == 3}

            possible_modules = {m for m in possible_modules if m in {"quality-module-3", "speed-module-3"}}
            possible_modules = set()

        
            for chosen_modules in chain.from_iterable(combinations_with_replacement(possible_modules, i) for i in range(slots)):

                key = (r["name"], q, m, chosen_modules)
                all_crafting_variables.add(key)

                effects = get_effect(chosen_modules)

                for k, i in enumerate(r.get("ingredients", [])):
                    ingredients_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add((key, k))
                for k, i in enumerate(r.get("results", [])):
                    if effects["quality"] > 0:
                        for qual in range(q, max_quality_level+1):
                            products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else qual)].add((key, k))
                    products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add((key, k))


In [5]:
import networkx as nx

G = nx.DiGraph()

# --- Add nodes ---

for item_key in all_item_variables:
    itype, iname, iq = item_key
    G.add_node(item_key, kind="item", itype=itype, name=iname, quality=iq)

for machine_key in all_crafting_variables:
    rname, q, m, mods = machine_key
    G.add_node(machine_key, kind="recipe", recipe=rname, quality=q, machine=m, mods=mods)

# --- Add ingredient edges: item -> recipe ---

for item_key in all_item_variables:
    for recipe_key, k in ingredients_to_recipes.get(item_key, []):
        rname, q, m, mods = recipe_key
        r = space_age["recipe"][rname]
        ing = r.get("ingredients", [])[k]

        amt = ing.get("amount", 0)
        if not amt:
            continue

        # item consumed per craft of recipe_key
        G.add_edge(
            item_key,
            recipe_key,
            kind="ingredient",
            per_machine=amt,
        )

# --- Add product edges: recipe -> item ---

alpha = 0.1  # 10% chance to upgrade again

def quality_prob(base_q, target_q, Q):
    # No quality modules or targeting below base quality
    if Q <= 0.0 or target_q < base_q:
        return 1.0 if target_q == base_q else 0.0

    if Q > 1.0:
        Q = 1.0

    # Already at max quality: can't go higher
    if base_q >= max_quality_level:
        return 1.0 if target_q == base_q else 0.0

    # Stay at base quality: no upgrade at all
    if target_q == base_q:
        return 1.0 - Q

    # Number of upgrade steps
    steps = target_q - base_q

    # Intermediate tiers: base_q < target_q < max_quality_level
    if target_q < max_quality_level:
        # P = Q * α^(steps-1) * (1-α)
        return Q * (alpha ** (steps - 1)) * (1.0 - alpha)

    # Top tier: target_q == max_quality_level
    # P = Q * α^(steps-1)
    if target_q == max_quality_level:
        return Q * (alpha ** (steps - 1))

    return 0.0

def clamp01(x):
    if x < 0.0:
        return 0.0
    if x > 1.0:
        return 1.0
    return x

for item_key in all_item_variables:
    itype, iname, iq = item_key

    for recipe_key, k in products_to_recipes.get(item_key, []):
        rname, q, m, mods = recipe_key
        r = space_age["recipe"][rname]
        res = r.get("results", [])[k]

        # base amount from amount / amount_min / amount_max
        if "amount" in res:
            min_amt = res["amount"]
            max_amt = res["amount"]
        else:
            min_amt = res.get("amount_min", 0)
            max_amt = res.get("amount_max", min_amt)
            if max_amt < min_amt:
                max_amt = min_amt
        base = 0.5 * (min_amt + max_amt)

        prob = res.get("probability", 1.0)
        extra = res.get("extra_count_fraction", 0.0)
        spoiled = res.get("percent_spoiled", 0.0)  # unused, same as your code

        effects = get_effect(mods)
        Q = clamp01(effects.get("quality", 0.0))

        prod_bonus = effects.get("productivity", 0.0)
        if not r.get("allow_productivity", False):
            prod_bonus = 0.0
        else:
            max_prod = r.get("maximum_productivity", 3.0)
            if prod_bonus < 0.0:
                prod_bonus = 0.0
            if prod_bonus > max_prod:
                prod_bonus = max_prod

        expected_base = (prob * base + extra) * (1.0 + prod_bonus)
        if expected_base == 0.0:
            continue

        if res["type"] == "fluid":
            # fluids ignore quality: all fluid is quality 0
            if iq != 0:
                continue
            expected = expected_base
        else:
            # items: quality distribution
            expected = expected_base * quality_prob(q, iq, Q)

        if not expected:
            continue

        # item produced per craft of recipe_key
        G.add_edge(
            recipe_key,
            item_key,
            kind="product",
            per_machine=expected,
        )


In [6]:
for u, v, data in sorted(G.in_edges(("item", "scrap", 0), data=True)):
    print(u, v, data)


ingredients_to_recipes[("item", "scrap", 0)]

{(('scrap-recycling', 0, ('furnace', 'recycler'), ()), 0)}

In [7]:
import gurobipy as grb

model = grb.Model()

A = [n for n, d in G.nodes(data=True) if d.get("kind") == "item"]
B = [n for n, d in G.nodes(data=True) if d.get("kind") == "recipe"]

total_machines = model.addVars(B, obj=100, vtype=GRB.INTEGER, name="machines")
machine_usage = model.addVars(B, name="usage")          # crafts per second
used = model.addVars(A, name="used")
produced = model.addVars(A, obj=1, name="produced")

# --- flow feasibility ---

model.addConstrs(used[i] <= produced[i] for i in A)

# --- capacity constraints (independent of per_machine) ---

for machine_key in B:
    rname, q, m, mods = machine_key
    machine = space_age[m[0]][m[1]]
    recipe = space_age["recipe"][rname]

    base_speed = machine.get("crafting_speed", 1.0)
    craft_time = recipe.get("energy_required", 0.5)

    effects = get_effect(mods)
    speed_mult = effects.get("speed", 1.0)

    eff_speed = base_speed * speed_mult
    crafts_per_second = eff_speed / craft_time

    model.addConstr(
        machine_usage[machine_key]
        <= total_machines[machine_key] * crafts_per_second
    )

# --- material balance using per_machine from the graph ---

from collections import defaultdict

expr_ingr = dict()
expr_resu = dict()

model.update()

for i in A:
    expr_u = grb.LinExpr()
    expr_p = grb.LinExpr()

    # item -> recipe edges (ingredients)
    for _, r, data in G.out_edges(i, data=True):
        if data.get("kind") != "ingredient":
            continue
        a = data["per_machine"]
        expr_ingr[(i, r)] = a * machine_usage[r]
        expr_u.addTerms(a, machine_usage[r])

    # recipe -> item edges (products)
    for r, _, data in G.in_edges(i, data=True):
        if data.get("kind") != "product":
            continue
        a = data["per_machine"]
        expr_resu[(r, i)] = a * machine_usage[r]
        expr_p.addTerms(a, machine_usage[r])

    # external sources/sinks (same as your original logic)
    if i == ("item", "scrap", 0):
        print("scrap")
        expr_p.addConstant(10000)
        produced[i].Obj = 0
    
    if i == ("fluid", "heavy-oil", 0):
        print("heavy oil")
        expr_p.addConstant(10000)
        produced[i].Obj = 0


    if i == ("item", "quality-module-3", 0):
        expr_p.addConstant(-0.1)


    model.addConstr(used[i] == expr_u, name=f"used-{i}")
    model.addConstr(produced[i] == expr_p, name=f"produced-{i}")

# model.setParam(GRB.Param.MIPGap, 0.04)
model.optimize()


Set parameter Username
Set parameter LicenseID to value 2732003
Academic license - for non-commercial use only - expires 2026-11-03
scrap
heavy oil
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 10948 rows, 14324 columns and 37027 nonzeros
Model fingerprint: 0x8a387efa
Variable types: 9055 continuous, 5269 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+04]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+04]
Presolve removed 5507 rows and 6190 columns
Presolve time: 0.03s
Presolved: 5441 rows, 8134 columns, 23268 nonzeros
Variable types: 4067 continuous, 4067 integer (0 binary)

Root relaxation: objective 1.261480e+04, 359 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

In [8]:
s = []
for i in A:
    if used[i].X > 0:
        s += [f"{(i, used[i], produced[i].X)}"]


for m in B:
    if total_machines[m].X > 0:
        print(m, total_machines[m].X)


('advanced-circuit', 0, ('assembling-machine', 'electromagnetic-plant'), ()) 4.0
('heavy-oil-cracking', 0, ('assembling-machine', 'chemical-plant'), ()) 1.0
('scrap-recycling', 0, ('furnace', 'recycler'), ()) 92.99999751328951
('quality-module-2', 0, ('assembling-machine', 'electromagnetic-plant'), ()) 6.0
('quality-module-3', 0, ('assembling-machine', 'electromagnetic-plant'), ()) 3.0
('ice-melting', 0, ('assembling-machine', 'chemical-plant'), ()) 1.0
('processing-unit-recycling', 0, ('furnace', 'recycler'), ()) 3.0
('holmium-solution', 0, ('assembling-machine', 'chemical-plant'), ()) 1.0
('low-density-structure-recycling', 0, ('furnace', 'recycler'), ()) 4.0
('holmium-plate', 0, ('assembling-machine', 'foundry'), ()) 1.0
('superconductor', 0, ('assembling-machine', 'electromagnetic-plant'), ()) 1.0
('advanced-circuit', 0, ('assembling-machine', 'assembling-machine-3'), ()) 1.0
('quality-module', 0, ('assembling-machine', 'electromagnetic-plant'), ()) 12.0


In [9]:
import gurobipy as grb
from collections import defaultdict

# Sets of nodes from G
A = [n for n, d in G.nodes(data=True) if d.get("kind") == "item"]
B = [n for n, d in G.nodes(data=True) if d.get("kind") == "recipe"]

# Macro solution
n_r = {r: int(round(total_machines[r].X)) for r in B}
u_r = {r: machine_usage[r].X for r in B}  # crafts per second (total for recipe r)

# --- Build machine instances ---

machine_instances = []          # list of (r, k)
machine_of_inst = {}            # (r,k) -> r

for r in B:
    n = n_r[r]
    if n <= 0:
        continue
    for k in range(n):
        inst = (r, k)
        machine_instances.append(inst)
        machine_of_inst[inst] = r

# Per-instance production/consumption of each item
prod_cap = dict()   # (inst, item) -> supply rate (units/s)
cons_req = dict()   # (inst, item) -> demand rate (units/s)

producers = defaultdict(list)   # item -> [inst]
consumers = defaultdict(list)   # item -> [inst]

for inst in machine_instances:
    r = machine_of_inst[inst]
    n = n_r[r]
    if n == 0 or u_r[r] == 0.0:
        continue

    u_inst = u_r[r] / n  # crafts/s on this instance (equal split)

    # Products: r -> i
    for _, i, data in G.out_edges(r, data=True):
        if data.get("kind") != "product":
            continue
        a = data["per_machine"]       # units per craft
        cap = a * u_inst              # units/s produced by this instance
        if cap > 0.0:
            prod_cap[(inst, i)] = cap
            producers[i].append(inst)

    # Ingredients: i -> r
    for i, _, data in G.in_edges(r, data=True):
        if data.get("kind") != "ingredient":
            continue
        a = data["per_machine"]       # units per craft
        req = a * u_inst              # units/s consumed by this instance
        if req > 0.0:
            cons_req[(inst, i)] = req
            consumers[i].append(inst)

# Items that are actually routed between machines
items_routed = [i for i in A if producers[i] and consumers[i]]

In [10]:
micro = grb.Model("micro_routing")

flows = {}   # (i, p, c) -> f variable
links = {}   # (i, p, c) -> y variable

# Big-M per item: total production of that item (upper bound on any single link)
M_item = {}
for i in items_routed:
    M_item[i] = sum(prod_cap[(p, i)] for p in producers[i])

# Create variables
for i in items_routed:
    for p in producers[i]:
        for c in consumers[i]:
            f = micro.addVar(lb=0.0, name="flow")
            y = micro.addVar(vtype=grb.GRB.BINARY)
            flows[(i, p, c)] = f
            links[(i, p, c)] = y

micro.update()

# --- Constraints ---

# Producer capacity: sum_c f[i, p, c] ≤ prod_cap[p, i]
for (p, i), cap in prod_cap.items():
    if i not in items_routed:
        continue
    relevant_cs = [c for c in consumers[i] if (i, p, c) in flows]
    if not relevant_cs:
        continue
    micro.addConstr(
        grb.quicksum(flows[(i, p, c)] for c in relevant_cs) <= cap,
        name=f"prod_cap_{p}_{i}",
    )

# Consumer demand: sum_p f[i, p, c] ≥ cons_req[c, i]
for (c, i), req in cons_req.items():
    if i not in items_routed:
        continue
    relevant_ps = [p for p in producers[i] if (i, p, c) in flows]
    if not relevant_ps:
        continue
    micro.addConstr(
        grb.quicksum(flows[(i, p, c)] for p in relevant_ps) >= req,
        name=f"cons_req_{c}_{i}",
    )

# Link activation: f[i, p, c] ≤ M_i * y[i, p, c]
for (i, p, c), f in flows.items():
    micro.addConstr(
        f <= M_item[i] * links[(i, p, c)]
    )

# --- Objective: minimize number of links ---

micro.setObjective(
    grb.quicksum(links.values()),
    sense=grb.GRB.MINIMIZE,
)

micro.write("micro.lp")
micro.setParam(GRB.Param.MIPGap, 0.1)
micro.optimize()


Set parameter MIPGap to value 0.1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
MIPGap  0.1

Optimize a model with 5309 rows, 9060 columns and 18120 nonzeros
Model fingerprint: 0xcd8b4b2d
Variable types: 4530 continuous, 4530 integer (4530 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 4e+00]
Found heuristic solution: objective 431.0000000
Presolve removed 3066 rows and 4818 columns
Presolve time: 0.09s
Presolved: 2243 rows, 4242 columns, 8484 nonzeros
Found heuristic solution: objective 428.0000000
Variable types: 2121 continuous, 2121 integer (2121 binary)

Root relaxation: interrupted, 3112 iterations, 0.02 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl

In [11]:
import networkx as nx

# H: micro-level machine graph
H = nx.MultiDiGraph()

machine_to_int = dict()
# 1. Add machine-instance nodes
for inst in machine_instances:
    r = machine_of_inst[inst]   # original recipe/machine node
    (rname, q, m, mods), k = inst
    H.add_node(len(machine_to_int), recipe=rname, quality=q, machine="/".join(m), modules=";".join(mods), index=k)
    
    machine_to_int[inst] = len(machine_to_int)

# 2. Add edges for positive flows
for (item_key, prod_inst, cons_inst), fvar in flows.items():
    flow = fvar.X

    if flow <= 0.0:
        continue

    # one edge per (item, producer, consumer)
    H.add_edge(
        machine_to_int[prod_inst],
        machine_to_int[cons_inst],
        item=item_key,
        flow=flow,
    )


In [ ]:
nx.write_gml(H, "data/micro.gml")

In [13]:
import fio.layout

fio.layout.compute_layers_multi(H, set(), set())

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 559 rows, 560 columns and 1546 nonzeros
Model fingerprint: 0xe4b3878a
Variable types: 0 continuous, 560 integer (428 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives (3 combined)...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
---------------------------------------------------------------------------

Presolve time: 0.00s
Presolved: 559 rows, 560 columns, 1546 nonzeros

Multi-objectives: optimize objective 1 (minimize feedback ) ...
----------------------

({0: 2,
  1: 2,
  2: 2,
  3: 2,
  4: 3,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 1,
  15: 0,
  16: 0,
  17: 0,
  18: 3,
  19: 0,
  20: 0,
  21: 0,
  22: 2,
  23: 0,
  24: 0,
  25: 0,
  26: 0,
  27: 0,
  28: 0,
  29: 1,
  30: 0,
  31: 0,
  32: 0,
  33: 0,
  34: 0,
  35: 0,
  36: 0,
  37: 0,
  38: 1,
  39: 0,
  40: 0,
  41: 0,
  42: 0,
  43: 0,
  44: 0,
  45: 0,
  46: 1,
  47: 0,
  48: 0,
  49: 0,
  50: 0,
  51: 0,
  52: 0,
  53: 0,
  54: 0,
  55: 0,
  56: 0,
  57: 0,
  58: 0,
  59: 2,
  60: 0,
  61: 0,
  62: 0,
  63: 0,
  64: 0,
  65: 0,
  66: 0,
  67: 0,
  68: 0,
  69: 0,
  70: 0,
  71: 0,
  72: 0,
  73: 0,
  74: 3,
  75: 0,
  76: 1,
  77: 0,
  78: 0,
  79: 0,
  80: 1,
  81: 0,
  82: 0,
  83: 0,
  84: 0,
  85: 2,
  86: 0,
  87: 0,
  88: 0,
  89: 0,
  90: 0,
  91: 0,
  92: 0,
  93: 0,
  94: 0,
  95: 0,
  96: 0,
  97: 0,
  98: 3,
  99: 4,
  100: 4,
  101: 4,
  102: 4,
  103: 3,
  104: 5,
  105: 5,
  106: 5,
  107: 1,
  108: 1,
  109: 1,
  110: 1,
